In [1]:
import os
import re
import csv
import sys
import ast 
import json
import time
import urllib
import random
import spotipy
import requests
import datetime
import numpy as np
import pandas as pd
import urllib.parse
from bs4 import BeautifulSoup
from user_agent import generate_user_agent
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
# spotifyAPI存取權限
client_id = '1a23ed1ca359485ab81d524206dba0f6'
client_secret = 'af0e14fdb19a49dfb19ef0c1ea1d42ea'
username = 'qnwv65t11cplaz4dikhl4mjgi'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
# 檔案名稱所有非法字元都轉義：https://www.itread01.com/content/1549125182.html
def validateTitle(title):
    rstr = r"[\/\\\:\*\?\"\<\>\|]" # '/ \ : * ? " < > |'
    new_title = re.sub(rstr, " ", title)  # 替換為空白
    return new_title

In [4]:
# 取得歌手的所有專輯
def show_artist_albums(artist_id):
    albums = []
    # album_type有四種：album、single、appears_on、compilation (include_groups=album,single)
    results = sp.artist_albums(artist_id, album_type='album')
    albums.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        albums.extend(results['items'])
    # 移除重複的album
    seen = set()
    # 按照專輯的名稱排序
    albums.sort(key=lambda album: album['name'].lower())
    list_album_name = []
    list_album_id = []
    list_album_uri = []
    list_album_href = []
    for album in albums:
        album_name = album['name']
        album_id = album['id']
        album_uri = album['uri']
        album_href = 'https://open.spotify.com/album/' + album_id
        if album_name not in seen:
            #print(album_name)
            seen.add(album_name)
            list_album_name.append(album_name)
            list_album_id.append(album_id)
            list_album_uri.append(album_uri)
            list_album_href.append(album_href)
    df_albums = pd.DataFrame(list(zip(list_album_name,list_album_id,list_album_uri,list_album_href)), columns =['Name','Id','Uri','Href'])
    df_albums.drop_duplicates(subset='Name', keep='first', inplace=True)
    df_albums.reset_index(drop=True, inplace=True)
    return df_albums

In [8]:
# 每位歌手抓取專輯資料並輸出csv(df_albums_+ artist_name +.csv)
empty_album = [] #沒有專輯的歌手
exist_album = [] #有專輯的歌手

df_id_genres = pd.read_csv(r'csv/20200303/v1/artist_list_ID+Genres_select_v1.csv')
for i in range(len(df_id_genres)):
    artist_name = df_id_genres['Artist'][i]
    artist_id = df_id_genres['Id'][i]
    
    df_albums = show_artist_albums(artist_id)
    # 歌手沒有專輯data
    if df_albums.empty == True:
        empty_album.append(artist_name) #沒有專輯data的歌手名稱放入empty_album列表中
        print(str(i) + ' | ' + str(artist_name) + ' | ' + 'N')
    # 歌手有專輯data
    if df_albums.empty == False:
        exist_album.append(artist_name) #有專輯data的歌手名稱放入exist_album列表中
        artist_name = validateTitle(artist_name) #檔名去除無效字元
        df_albums.to_csv('csv/20200303/v1/v1_albums/df_albums_'+ artist_name +'.csv',index=0,encoding="utf_8_sig")
        print(str(i) + ' | ' + str(artist_name) + ' | ' + 'Y')

print('DONE')

0 | 5 Seconds of Summer | Y
1 | Aaron Carter | Y
2 | Adam Rickitt | Y
3 | Agnetha Faltskog | Y
4 | Agnez Mo | Y
5 | Ailee | Y
6 | AJ Mitchell | N
7 | Al Jolson | Y
8 | Alan Walker | Y
9 | Alannah Myles | Y
10 | Alec Benjamin | Y
11 | Alisan Porter | Y
12 | Aly Michalka | N
13 | Amber Riley | N
14 | Anastacia | Y
15 | Anne Murray | Y
16 | Anne-Marie | Y
17 | Ariana Grande | Y
18 | Ariel Camacho | Y
19 | Ashanti | Y
20 | Ashley Tisdale | Y
21 | Astrid S | N
22 | Avril Lavigne | Y
23 | Ayo Jay | Y
24 | Bad Bunny | Y
25 | Bazzi | Y
26 | Beyoncé | Y
27 | Bilal Hassani | Y
28 | Birdy | Y
29 | Bob Geldof | Y
30 | Bobby Brown | Y
31 | Brendon Urie | N
32 | Brian Littrell | Y
33 | Bridgit Mendler | Y
34 | Britney Spears | Y
35 | Britt Nicole | Y
36 | Bruno Mars | Y
37 | Calum Scott | Y
38 | Calvin Harris | Y
39 | Camila Cabello | Y
40 | Carly Rae Jepsen | Y
41 | Chad Kroeger | Y
42 | Charlie Puth | Y
43 | Charlotte Gainsbourg | Y
44 | Charlotte Lawrence | N
45 | Cheyenne Jackson | Y
46 | Christ